## 個人課題１
- Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
    - リポジトリ名
    - 主要な言語
    - スターの数
- 上記のデータを保存するためのDBを作成し，スクレイピングしたデータを保存する
    - GithubのAPIは使わないこと（スクレイピングの課題なので）
    - ちゃんと`time.sleep(1)`入れてね
- 保存したデータをSELECT文で表示する

In [4]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time

In [9]:
# 取得したリポジトリを保存するデータベースのテーブルを作成
path = ''
db_name = 'repositories.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RUBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # テーブルの作成
    sql = 'CREATE TABLE repositories(id INTEGER PRIMARY KEY, name TEXT, language TEXT, stars REAL);'

    # SQL文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました：', e)

finally:
    # DBへの接続を閉じる
    conn.close()

In [10]:
path = ''
db_name = 'repositories.db'

total = 0

# aタグの中のaria-labelにstarsが含まれている場合にTrueを返す関数
def has_stars(tag):
    # a タグでなければスキップ
    if tag.name != "a":
        return False
    aria = tag.get("aria-label")
    return aria is not None and "stars" in aria

# 全てのリポジトリを取得するために全てのページにアクセスする
for page in range(1, 95):
    time.sleep(1)
    url = f"https://github.com/orgs/google/repositories?page={page}"
    response = requests.get(url)
    # ステータスコードが200以外の場合に例外を発生させる
    response.raise_for_status()
    print("ステータスコード:", response.status_code)
    print("-----")
    response.encoding = response.apparent_encoding 
    soup = BeautifulSoup(response.text, "html.parser")

    # リポジトリのブロックを取得するコードをかく
    # 各リポジトリが一つの<li>タグとしてまとまっているのでそれを取得
    repo_blocks = soup.find_all('li', class_="ListItem-module__listItem--k4eMk")
    for repo_block in repo_blocks:
        time.sleep(1)

        # リポジトリの名前を取得
        # aタグの中の指定したクラスの中にあるテキストを取得する
        a_tag = repo_block.find('a', class_="Title-module__anchor--GmXUE Title-module__inline--oM0P7")
        if a_tag:
            repo_name = a_tag.get_text()
        else:
            repo_name = None
        # print(repo_name)

        # 主要な言語の取得
        # spanタグの中の指定したクラスの中にあるテキストを取得する
        language_tags = repo_block.find('span', class_='ReposListItem-module__Text_4--mkG7R')
        if language_tags:
            language = language_tags.get_text()
        else:
            language = None
        # print(language)

        # スターの数の取得
        star_tags = repo_block.find(has_stars)
        if star_tags:
            stars_num = star_tags.get_text()  
        else:
            stars_num = None
        # print(stars_num)

        # 取得したリポジトリの名前と主要な言語とスターの数をデータベースに保存
        try:
            # DB接続オブジェクトの作成
            conn = sqlite3.connect(path + db_name)

            # SQL(RUBを操作するための言語)を実行するためのカーソルp武ジェクトを取得
            cur = conn.cursor()

            # データ挿入のSQL文
            sql = "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?);"

            repositories = [
                (repo_name, language, stars_num)
            ]

            cur.executemany(sql, repositories)

            # 変更をDBに反映させる
            conn.commit()
            
        except sqlite3.Error as e:
            print('エラーが発生しました：', e)

        finally:
            # DBへの接続を閉じる
            conn.close()
        
        total += 1
    # 途中経過の表示
    print(f"【ページ {page} 終了】 現在までの取得件数：{total} 件")
    print("-----")

print("==== 全ページ取得完了 ====")
print(f"総件数：{total} 件")

ステータスコード: 200
-----
【ページ 1 終了】 現在までの取得件数：30 件
-----
ステータスコード: 200
-----
【ページ 2 終了】 現在までの取得件数：60 件
-----
ステータスコード: 200
-----
【ページ 3 終了】 現在までの取得件数：90 件
-----
ステータスコード: 200
-----
【ページ 4 終了】 現在までの取得件数：120 件
-----
ステータスコード: 200
-----
【ページ 5 終了】 現在までの取得件数：150 件
-----
ステータスコード: 200
-----
【ページ 6 終了】 現在までの取得件数：180 件
-----
ステータスコード: 200
-----
【ページ 7 終了】 現在までの取得件数：210 件
-----
ステータスコード: 200
-----
【ページ 8 終了】 現在までの取得件数：240 件
-----
ステータスコード: 200
-----
【ページ 9 終了】 現在までの取得件数：270 件
-----
ステータスコード: 200
-----
【ページ 10 終了】 現在までの取得件数：300 件
-----
ステータスコード: 200
-----
【ページ 11 終了】 現在までの取得件数：330 件
-----
ステータスコード: 200
-----
【ページ 12 終了】 現在までの取得件数：360 件
-----
ステータスコード: 200
-----
【ページ 13 終了】 現在までの取得件数：390 件
-----
ステータスコード: 200
-----
【ページ 14 終了】 現在までの取得件数：420 件
-----
ステータスコード: 200
-----
【ページ 15 終了】 現在までの取得件数：450 件
-----
ステータスコード: 200
-----
【ページ 16 終了】 現在までの取得件数：480 件
-----
ステータスコード: 200
-----
【ページ 17 終了】 現在までの取得件数：510 件
-----
ステータスコード: 200
-----
【ページ 18 終了】 現在までの取得件数：540 件
-----
ステータスコード: 200
-----
【ページ 19 終了】 現在までの取得件

In [11]:
# データベースに保存したデータをSELECT文を用いて表示
path = ''
db_name = 'repositories.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RUBを操作するための言語)を実行するためのカーソルp武ジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # SELECT * FROM テーブル名;
    # *　の部分は取得したい列の名前を区切りで指定することができる
    sql = "SELECT * FROM repositories;"

    # SQL文の実行
    cur.execute(sql)
    
except sqlite3.Error as e:
    print('エラーが発生しました：', e)

else:
    for row in cur:
        # 行データ(row)はタプルなので、アンバックして列データを取得
        id, name, language, stars = row
        print(row)

finally:
    # DBへの接続を閉じる
    conn.close()

(1, 'tunix', 'Python', '1.9k')
(2, 'device-infra', 'Java', 58.0)
(3, 'dive', 'C++', 17.0)
(4, 'nomulus', 'Java', '1.8k')
(5, 'jetpack-camera-app', 'Kotlin', 275.0)
(6, 'gvisor', 'Go', '17k')
(7, 'quiche', 'C++', 801.0)
(8, 'cel-java', 'Java', 227.0)
(9, 'dawn', 'C++', 770.0)
(10, 'XNNPACK', 'C', '2.2k')
(11, 'j2cl', 'Java', '1.3k')
(12, 'xls', 'C++', '1.4k')
(13, 'orbax', 'Python', 454.0)
(14, 'ml-metrics', 'Python', 18.0)
(15, 'perfetto', 'C++', '5k')
(16, 'automotive-design-compose', 'Rust', 164.0)
(17, 'TestParameterInjector', 'Java', 415.0)
(18, 'adk-python', 'Python', '15k')
(19, 'grain', 'Python', 601.0)
(20, 'meridian', 'Python', '1.2k')
(21, 'docsy', 'JavaScript', '2.9k')
(22, 'heir', 'C++', 605.0)
(23, 'init2winit', 'Python', 79.0)
(24, 'langfun', 'Python', 875.0)
(25, 'filament', 'C++', '19k')
(26, 'chromium-policy-vulnfeed', 'Go', 7.0)
(27, 'bloaty', 'C++', '5.3k')
(28, 's2geometry', 'C++', '2.6k')
(29, 'pullsheet', 'Go', 88.0)
(30, 'nearby', 'C++', 881.0)
(31, 'nsscache', '